In [418]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.

In [419]:
# Загружаем датасет iris, определяем признаки и целевую переменную
iris = load_iris()
X = iris.data
y = iris.target

In [420]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [421]:
# Оставляем 2 класса: 0 - 'versicolor', 1 - 'virginica'
X = iris.data[iris.target != 0]
y = iris.target[iris.target != 0] - 1
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [422]:
X[:, 1]

array([3.2, 3.2, 3.1, 2.3, 2.8, 2.8, 3.3, 2.4, 2.9, 2.7, 2. , 3. , 2.2,
       2.9, 2.9, 3.1, 3. , 2.7, 2.2, 2.5, 3.2, 2.8, 2.5, 2.8, 2.9, 3. ,
       2.8, 3. , 2.9, 2.6, 2.4, 2.4, 2.7, 2.7, 3. , 3.4, 3.1, 2.3, 3. ,
       2.5, 2.6, 3. , 2.6, 2.3, 2.7, 3. , 2.9, 2.9, 2.5, 2.8, 3.3, 2.7,
       3. , 2.9, 3. , 3. , 2.5, 2.9, 2.5, 3.6, 3.2, 2.7, 3. , 2.5, 2.8,
       3.2, 3. , 3.8, 2.6, 2.2, 3.2, 2.8, 2.8, 2.7, 3.3, 3.2, 2.8, 3. ,
       2.8, 3. , 2.8, 3.8, 2.8, 2.8, 2.6, 3. , 3.4, 3.1, 3. , 3.1, 3.1,
       3.1, 2.7, 3.2, 3.3, 3. , 2.5, 3. , 3.4, 3. ])

Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. 

In [423]:
# Для начала реализуем встроенный метод логистической регрессии

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучим модель встроенным модулем
model = LogisticRegression()
model.fit(X_train, y_train)

# И оценим качество модели
score = model.score(X_test, y_test)
print("Оценка точности модели:", score)

Оценка точности модели: 0.95


In [424]:
# Получим коэффициенты
model.coef_, model.intercept_

(array([[-0.35674468, -0.73750193,  2.83425127,  2.19287166]]),
 array([-13.16274078]))

In [425]:
y_pred = model.predict(X_test)
data_results = pd.DataFrame({
    'Реальные значения': y_test,
    'Предсказанные значения': y_pred
})
print(data_results)

    Реальные значения  Предсказанные значения
0                   1                       1
1                   1                       1
2                   1                       1
3                   0                       0
4                   0                       0
5                   0                       0
6                   0                       0
7                   1                       1
8                   0                       0
9                   0                       0
10                  0                       0
11                  0                       0
12                  1                       1
13                  0                       1
14                  1                       1
15                  0                       0
16                  1                       1
17                  1                       1
18                  0                       0
19                  0                       0


In [426]:
# Теперь самостоятельно реализуем метод логистической регрессии методом градиентного спуска

# Масштабируем признаки 
scaler = StandardScaler()
X_scaled_features = scaler.fit_transform(X)

# Добавляем столбец единиц для свободного веса
X = np.c_[np.ones(len(X_scaled_features)), X_scaled_features]

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Функция сигмоиды
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Функция для вычисления логарифмической функции потерь
def logloss(y, y_proba):
    return -np.mean(y * np.log(y_proba + 1e-30) + (1 - y) * np.log(1 - y_proba + 1e-30))

# Градиентный спуск
def gr_logloss(X, W, y):
    y_proba = sigmoid(X @ W)
    grad = X.T @ (y_proba - y) / len(y)
    return grad

# Параметры обучения 
# learning_rate: скорость обучения, epochs: количество итераций, beta: скорость сглаживания
eps = 0.0001
learning_rate = 0.001
beta1 = 0.9
beta2 = 0.999
epochs = 10000

# Инициализация весов
W = np.zeros(X.shape[1])

for i in range(epochs):
    cur_W = W.copy()  # текущие веса
    
    # Градиент по текущим весам
    grad = gr_logloss(X, cur_W, y)
    
    # Обновляем веса
    W_next = cur_W - learning_rate * grad
    
    # Проверка условия остановки
    if np.linalg.norm(W - W_next) <= eps:
        print(f"Остановились на итерации {i}")
        break
    
    # Обновляем веса для следующей итерации
    W = W_next
    
    # Каждые 100 итераций выводим информацию
    if i % 100 == 0:
        y_proba = sigmoid(X @ W)
        y_class = (y_proba >= 0.5).astype(int)
        accuracy = (y_class == y).mean()
        print(f"Итерация {i}")
        print(f"Logloss: {logloss(y, y_proba):.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print("--------------------------------------------------------")

Итерация 0
Logloss: 0.6927
Accuracy: 0.9000
--------------------------------------------------------
Итерация 100
Logloss: 0.6544
Accuracy: 0.9000
--------------------------------------------------------
Итерация 200
Logloss: 0.6210
Accuracy: 0.9000
--------------------------------------------------------
Итерация 300
Logloss: 0.5917
Accuracy: 0.9100
--------------------------------------------------------
Итерация 400
Logloss: 0.5659
Accuracy: 0.9100
--------------------------------------------------------
Итерация 500
Logloss: 0.5432
Accuracy: 0.9100
--------------------------------------------------------
Итерация 600
Logloss: 0.5230
Accuracy: 0.9100
--------------------------------------------------------
Итерация 700
Logloss: 0.5050
Accuracy: 0.9100
--------------------------------------------------------
Итерация 800
Logloss: 0.4887
Accuracy: 0.9300
--------------------------------------------------------
Итерация 900
Logloss: 0.4741
Accuracy: 0.9300
-----------------------------

In [427]:
data_results = pd.DataFrame({
    'Реальные значения': y,
    'Предсказанные значения': y_class,
    'Предсказанные вероятности': y_proba
})
print(data_results)

    Реальные значения  Предсказанные значения  Предсказанные вероятности
0                   0                       0                   0.298119
1                   0                       0                   0.256167
2                   0                       0                   0.424590
3                   0                       0                   0.073151
4                   0                       0                   0.307064
..                ...                     ...                        ...
95                  1                       1                   0.921544
96                  1                       1                   0.714147
97                  1                       1                   0.813697
98                  1                       1                   0.921004
99                  1                       1                   0.620383

[100 rows x 3 columns]


In [428]:
print("Финальные веса:", W)
i_grad = i
print("Итераций:", i_grad)
accuracy_grad = accuracy
print("Оценка точности модели методом градиентного спуска:", accuracy_grad)

Финальные веса: [ 0.02372129  0.26548554 -0.06459134  1.11430539  1.28339433]
Итераций: 8846
Оценка точности модели методом градиентного спуска: 0.95


Обучите логистическую регрессию этим методом скользящего среднего (Root Mean Square Propagation, RMSProp)

In [429]:
# Реализуем метод логистической регрессии методом скользящего среднего

eps = 0.001
W = np.zeros(X.shape[1])
# Инициализация переменной для скользящего среднего квадрата градиентов
v = np.zeros_like(W)

for i in range(epochs):
    cur_W = W.copy()  # текущие веса
    
    # Градиент по текущим весам
    grad = gr_logloss(X, cur_W, y)
    
    # Обновляем v
    v = beta1 * v + (1 - beta1) * (grad ** 2)
    
    # Обновляем веса с учетом RMSProp
    W_next = cur_W - (learning_rate / (np.sqrt(v) + eps)) * grad
    
    # Проверка условия остановки
    if np.linalg.norm(W - W_next) <= eps:
        print(f"Остановились на итерации {i}")
        break       
    
    # Обновляем веса для следующей итерации
    W = W_next

    # Каждые 100 итераций выводим информацию
    if i % 100 == 0:
        y_proba = sigmoid(X @ W)
        y_class = (y_proba >= 0.5).astype(int)
        accuracy = (y_class == y).mean()
        print(f"Итерация {i}")
        print(f"Logloss: {logloss(y, y_proba):.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print("--------------------------------------------------------")

Итерация 0
Logloss: 0.6894
Accuracy: 0.8000
--------------------------------------------------------
Итерация 100
Logloss: 0.5800
Accuracy: 0.8000
--------------------------------------------------------
Итерация 200
Logloss: 0.5045
Accuracy: 0.8000
--------------------------------------------------------
Итерация 300
Logloss: 0.4494
Accuracy: 0.8100
--------------------------------------------------------
Итерация 400
Logloss: 0.4057
Accuracy: 0.8400
--------------------------------------------------------
Итерация 500
Logloss: 0.3692
Accuracy: 0.8900
--------------------------------------------------------
Итерация 600
Logloss: 0.3387
Accuracy: 0.9100
--------------------------------------------------------
Итерация 700
Logloss: 0.3123
Accuracy: 0.9400
--------------------------------------------------------
Итерация 800
Logloss: 0.2887
Accuracy: 0.9400
--------------------------------------------------------
Итерация 900
Logloss: 0.2675
Accuracy: 0.9500
-----------------------------

In [430]:
print("Финальные веса:", W)
i_RMSProp = i
print("Итераций:", i_RMSProp)
accuracy_RMSProp = accuracy
print("Оценка точности модели методом скользящего среднего:", accuracy_RMSProp)

Финальные веса: [-0.09879358 -1.2334072  -1.44572217  5.18904281  5.15859536]
Итераций: 5763
Оценка точности модели методом скользящего среднего: 0.98


Обучите логистическую регрессию этим методом адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam)

In [431]:
# Реализуем метод логистической регрессии методом адаптивной оценки моментов

eps = 1e-8
W = np.zeros(X.shape[1])
# Инициализация переменных
m = np.zeros_like(W)
v = np.zeros_like(W)
t = 0

for i in range(epochs):
    t += 1

    grad = gr_logloss(X, cur_W, y)
    
    # Обновляем моменты
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * (grad ** 2)
    
    # Корректируем смещения для моментов
    m_hat = m / (1 - beta1 ** t)
    v_hat = v / (1 - beta2 ** t)
    
    # Предсказанный момент для Nesterov
    m_bar = beta1 * m_hat + (1 - beta1) * grad / (1 - beta1 ** t)
    
    # Обновляем веса
    W_next = cur_W - learning_rate * m_bar / (np.sqrt(v_hat) + eps)
    
    # Проверка условия остановки
    if np.linalg.norm(W - W_next) <= eps:
        print(f"Остановились на итерации {i}")
        break       
    
    # Обновляем веса для следующей итерации
    W = W_next

    # Каждые 10 итераций выводим информацию
    if i % 10 == 0:
        y_proba = sigmoid(X @ W)
        y_class = (y_proba >= 0.5).astype(int)
        accuracy = (y_class == y).mean()
        print(f"Итерация {i}")
        print(f"Logloss: {logloss(y, y_proba):.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print("--------------------------------------------------------")

Итерация 0
Logloss: 0.0639
Accuracy: 0.9800
--------------------------------------------------------
Итерация 10
Logloss: 0.0639
Accuracy: 0.9800
--------------------------------------------------------
Итерация 20
Logloss: 0.0639
Accuracy: 0.9800
--------------------------------------------------------
Итерация 30
Logloss: 0.0639
Accuracy: 0.9800
--------------------------------------------------------
Итерация 40
Logloss: 0.0639
Accuracy: 0.9800
--------------------------------------------------------
Итерация 50
Logloss: 0.0639
Accuracy: 0.9800
--------------------------------------------------------
Итерация 60
Logloss: 0.0639
Accuracy: 0.9800
--------------------------------------------------------
Итерация 70
Logloss: 0.0639
Accuracy: 0.9800
--------------------------------------------------------
Остановились на итерации 74


In [432]:
print("Финальные веса:", W)
i_Nadam = i
print("Итераций:", i_Nadam)
accuracy_Nadam = accuracy
print("Оценка точности модели методом адаптивной оценки моментов:", accuracy_Nadam)

Финальные веса: [-0.0997935  -1.23440719 -1.44672217  5.19004285  5.1595954 ]
Итераций: 74
Оценка точности модели методом адаптивной оценки моментов: 0.98


Сравните значение метрик для реализованных методов оптимизации. Напишите вывод.

In [433]:
data = {
    'Метод': ['Логистическая регрессия', 'Градиентный спуск', 'Метод скользящего среднего', 'Метод Нестерова'],
    'Метрика': [score, accuracy_grad, accuracy_RMSProp, accuracy_Nadam],
    'Время работы (итераций)': [1, i_grad, i_RMSProp, i_Nadam]
}

In [434]:
data = pd.DataFrame(data)
print(data)

                        Метод  Метрика  Время работы (итераций)
0     Логистическая регрессия     0.95                        1
1           Градиентный спуск     0.95                     8846
2  Метод скользящего среднего     0.98                     5763
3             Метод Нестерова     0.98                       74


### Итоги:
На выходе мы имеем хорошие показатели метрики оценки качества модели по всем используемым методам. Метод Нестерова показывает самый быстрый расчет по сравнению с Градиентным спуском и Скользящим средним. Наилучшие показатели точности у Метода скользящего среднего и Нестерова. Подводя итог, можно сделать вывод, что скорость и полученная метрика зависят от настроек параметров. В частности, варьируя показателями скорости обучения и количеством итераций можно повысить метрику качества для Градиентного спуска до 0.97 (learning_rate = 0.01, epochs = 100000).

PS. Стоит также отметить, что в написанных функциях обучение происходит на всей выборке X (а в модуле LogisticRegression мы используем тренировочную выборку)